## Import Dependencies 

In [1]:
import sys

# sys.path.insert(0, '../MIR')

In [3]:
from src.utils.construct_positional_indexes import construct_positional_indexes
from src.utils.load_index import load_index
from src.models.TextPreparer import TextPreparer
from src.enums import *

In [4]:
manager = construct_positional_indexes()
manager.save_index("manager.pickle")

100%|██████████| 1572/1572 [00:14<00:00, 109.24it/s]


In [3]:
manager = load_index("manager.pickle")

In [4]:
manager.corpus_index.get_posting_list("با")[0].get_positions(Fields.TEXT)

[278, 313, 330, 477]

In [6]:
word = "آقو"

In [7]:
manager.correct_word(word)

'آقوس'

In [9]:
manager.search("نیمه شعبان")

[5619,
 3666,
 5236,
 6208,
 7123,
 6177,
 3665,
 6791,
 3360,
 6475,
 4438,
 7101,
 6626,
 6548,
 4335]

In [10]:
manager.search("نیمه شعبان")

[5619,
 3666,
 5236,
 6208,
 7123,
 6177,
 3665,
 6791,
 3360,
 6475,
 4438,
 7101,
 6626,
 6548,
 4335]

In [5]:
manager.search("نیمه شعبان")

[5619,
 3666,
 5236,
 6208,
 7123,
 6177,
 3665,
 6791,
 3360,
 6475,
 4438,
 7101,
 6626,
 6548,
 4335]

In [17]:
manager.add_document_to_indexes("./data/Add.xml", 7157)

In [15]:
manager.search({Fields.TITLE: '"آقا علیرضا"'})

[]

In [14]:
manager.delete_document_from_indexes("./data/Add.xml", 7157)

In [18]:
print(manager.corpus_index.index["آقا"].posting_list[-1])


        Document ID: 7157
        Title Positions: [0]
        Text Positions: [0]
        


In [19]:
manager.corpus_index.index["آقا"].doc_frequency

{<Fields.TITLE: 'title'>: 1, <Fields.TEXT: 'text'>: 62}

In [20]:
from src.metrics import (
    f_measure,
    precision,
    r_precision,
    recall,
    ndcg_at_k,
    average_precision,
    evaluate_search_engine,
)

In [22]:
evaluate_search_engine(manager, Methods.LTN_LNN)

r_precision: 0.6105804602529432
ndcg: 0.6787020126128037
f_measure: 0.6279770638289564
map: 0.7569273771354816
